# Setup

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F

# Hard code the data and seperate into x and y

In [2]:
train_data = torch.tensor(
    [
        [1,1,0],
        [1,0,1],
        [0,1,1],
        [0,0,1]
    ],
    dtype=torch.float
)

x, y = train_data[:, :2], train_data[:, -1]
y = y.view(len(y), 1)

# Create a Module 
it has an input of two with 2 hidden units that are converted using the sigmoid function into 1 final unit

In [3]:
class NAND(nn.Module):           
  def __init__(self, D_in, D_out):
    super(NAND, self).__init__()
    self.lin1 = nn.Linear(D_in, 2, True)
    self.lin2 = nn.Linear(2, D_out, True)
  
  def forward(self, y):
    y = self.lin1(y)
    y = torch.sigmoid(y)
    y = self.lin2(y)
    return y

model = NAND(2, 1)

# Create loss and Optimize function

In [4]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)                 

# Run through 1000 epochs
the loss on the last epoch was only 0.050379

In [5]:
epochs = 1000

for epoch in range(epochs):
  y_pred = model(x)
  loss = criterion(y_pred, y)
  if (epoch + 1) % 100 == 0:
    print('Epoch:', epoch + 1, '\tLoss:', loss.item())

  optimizer.zero_grad()                             
  loss.backward()                                    
  optimizer.step()                                    

Epoch: 100 	Loss: 0.18775887787342072
Epoch: 200 	Loss: 0.17845693230628967
Epoch: 300 	Loss: 0.15158870816230774
Epoch: 400 	Loss: 0.10943273454904556
Epoch: 500 	Loss: 0.07974863797426224
Epoch: 600 	Loss: 0.0641881674528122
Epoch: 700 	Loss: 0.05522724613547325
Epoch: 800 	Loss: 0.048910416662693024
Epoch: 900 	Loss: 0.043881095945835114
Epoch: 1000 	Loss: 0.03964855521917343


# Print out Predictions

In [6]:
x_dev = torch.tensor(
    [
        [1, 1],
        [1, 0],
        [0, 1],
        [0, 0]
    ],
    dtype=torch.float
)

with torch.no_grad():
  y_dev_pred = model(x_dev)

y_dev_pred = torch.round(y_dev_pred)

for x, y in zip(x_dev, y_dev_pred):
    print(x, ' prediction: ', y)

tensor([1., 1.])  prediction:  tensor([0.])
tensor([1., 0.])  prediction:  tensor([1.])
tensor([0., 1.])  prediction:  tensor([1.])
tensor([0., 0.])  prediction:  tensor([1.])


In [7]:
import anvil.server

anvil.server.connect("Q7HVYD4QARNLXM3W46335F4N-DJV5KJZGS5LSOGW2")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [8]:
@anvil.server.callable

def NANDFunc(input):
    input = torch.tensor(
        [
            input
        ],
        dtype = torch.float
    )
    print(input)
    
    with torch.no_grad():
        output = model(input )
    
    print(output)
    output = torch.round(output)
    output = output.numpy()[0][0]
    return output

tensor([[1., 1.]])
tensor([[0.1876]])
tensor([[1., 0.]])
tensor([[0.8006]])
tensor([[1., 1.]])
tensor([[0.1876]])
tensor([[0., 0.]])
tensor([[1.2126]])
